In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

In [2]:
def frame_array(video, info):
    all_frames = []
    cap = cv2.VideoCapture(os.path.join('videos',video))
    fps = cap.get(cv2.CAP_PROP_FPS)
    dimensions = (int(cap.get(3)),int(cap.get(4)))
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        """try:
            frame = cv2.resize(frame, (200,200))
        except:
            break"""
        all_frames.append(frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Close down everything
    if info:
        dimensions = (int(cap.get(3)),int(cap.get(4)))
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        cv2.destroyAllWindows()
        return all_frames, fps, dimensions
    else:
        return all_frames

In [3]:
# workout
# play 10 second timer
# play workout video
# follow along
# show similarity score in a popup next to it
# save similarity score and live video
# point out mistakes and differences

def workout(instruct_vid): #, output_name
    # cap1 = cv2.VideoCapture(os.path.join('videos','10secCountdown.mp4'))
    # cap2 = cv2.VideoCapture(os.path.join('videos','jumpingjacks.mp4'))
    all_frames_vid1 = frame_array('10secCountdown.mp4', False)
    all_frames_vid2, fps2, dimensions2 = frame_array(instruct_vid, True)
    
    # show 10 sec countdown
    for frame1 in all_frames_vid1:
        cv2.imshow("Video Player", frame1)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    # show live video at same time as workout video
    cap_live = cv2.VideoCapture(0)
    fps_live = cap_live.get(cv2.CAP_PROP_FPS)
    dimensions_live = (int(cap_live.get(3)),int(cap_live.get(4)))
    """
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap_live.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(os.path.join('videos','outputs',output_name), 
                                   fourcc, fps, (int(cap_live.get(3)),int(cap_live.get(4)))) """
    live_frames = [0] * len(all_frames_vid2)
    for frame_idx, frame2 in enumerate(all_frames_vid2):
        cv2.imshow('Video Player', frame2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        ret_live, frame_live = cap_live.read()
        live_frames[frame_idx] = frame_live
        # video_writer.write(frame_live)    
    
    cap_live.release()
    cv2.destroyAllWindows()
    return all_frames_vid2, live_frames, fps2, fps_live, dimensions2, dimensions_live

In [5]:
workout("cutjumpjacks360.mp4")

([], [], 0.0, 1.0, (0, 0), (1280, 720))

In [4]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle

In [6]:
# nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, 
# left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle

EDGES = {
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [8]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    
    return angle

In [44]:
def frames_comparison(all_frames1, all_frames2, fps1, fps2, output1, output2, dimensions1, dimensions2):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer1 = cv2.VideoWriter(os.path.join('videos','outputs',output1), 
                                   fourcc, fps1, dimensions1)
    video_writer2 = cv2.VideoWriter(os.path.join('videos','outputs',output2), 
                                   fourcc, fps2, dimensions2)
    for frame_idx, frame1 in enumerated(all_frames1):
        frame2 = all_frames2[frame_idx]
        if (frame1 is None) or (frame2 is None):
            break
        img1 = frame1.copy()
        img1 = tf.image.resize_with_pad(tf.expand_dims(img1, axis = 0), 192, 192)
        input_image1 = tf.cast(img1, dtype=tf.float32)
        
        img2 = frame2.copy()
        img2 = tf.image.resize_with_pad(tf.expand_dims(img2, axis = 0), 192, 192)
        input_image2 = tf.cast(img2, dtype=tf.float32)

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image1))
        interpreter.invoke()
        keypoints_with_scores1 = interpreter.get_tensor(output_details[0]['index'])

        wanted_scores1 = keypoints_with_scores1[0][0][5:]
        draw_connections(frame1, keypoints_with_scores1, EDGES, 0.6)
        draw_keypoints(frame1, wanted_scores1, 0.6)
        
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image2))
        interpreter.invoke()
        keypoints_with_scores2 = interpreter.get_tensor(output_details[0]['index'])

        wanted_scores2 = keypoints_with_scores2[0][0][5:]
        draw_connections(frame2, keypoints_with_scores2, EDGES, 0.6)
        draw_keypoints(frame2, wanted_scores2, 0.6)

        video_writer1.write(frame1) 
        video_writer2.write(frame2)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cv2.destroyAllWindows()
    return wanted_scores

In [45]:
def full_process():
    vid_frames, live_frames, fps_vid, fps_live = workout('cutjumpjacks360.mp4')
    scores_vid = add_keypoints_frames(vid_frames, "instructWkeypoints.mp4", fps_vid)
    scores_live = add_keypoints_frames(vid_frames, "instructWkeypoints.mp4", fps_vid)

In [46]:
def horiz_merge_two_videos(video1, video2):
    cap1 = cv2.VideoCapture(os.path.join('videos',video1))
    cap2 = cv2.VideoCapture(os.path.join('videos',video2))
    
    frame_width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap1.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, frame_rate, (frame_width * 2, frame_height))
    
    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break
        frame2 = cv2.resize(frame2, (frame_width, frame_height))
        canvas = np.zeros((frame_height, frame_width * 2, 3), dtype = np.uint8)
        canvas[:, :frame_width] = frame1
        canvas[:, frame_width:] = frame2
        out.write(canvas)
    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()
    return

In [47]:
def vert_merge_two_videos(video1, video2, output_name):
    cap1 = cv2.VideoCapture(os.path.join('videos',video1))
    cap2 = cv2.VideoCapture(os.path.join('videos',video2))
    
    frame_width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap1.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    full_frame_height = int(frame_height * 1.5)
    bottom_frame_height = full_frame_height - frame_height
    out = cv2.VideoWriter(output_name, fourcc, fps, (frame_width, full_frame_height))
    
    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break
        frame2 = cv2.resize(frame2, (frame_width, bottom_frame_height))
        canvas = np.zeros((full_frame_height, frame_width, 3), dtype = np.uint8)
        canvas[:frame_height, :] = frame1
        canvas[frame_height:, :] = frame2
        out.write(canvas)
    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()
    return

In [28]:
vert_merge_two_videos("cutjumpjacks360.mp4", 'jumpingjacks.mp4', 'output.mp4')